## Introduction ML

### Lab

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Part 1. Scikit-learn basics

#### 1. Estimators

In [2]:
#Consider toy dataset
X = np.array([[ 1,  2,  3], [11, 12, 13]])
y = np.array([2.1, 11.9])
print(X.shape, y.shape)

X_test =  np.array([[ 2, 3, 4]])
print(X_test.shape)

(2, 3) (2,)
(1, 3)


In [3]:
from sklearn.linear_model import LinearRegression

# Init the model and set all the hyperparameters (if there are any)
lr = LinearRegression()

# Train the model
lr.fit(X, y)
# make a prediction
lr.predict(X_test)

array([3.08])

#### 2. Transformer

In [4]:
from sklearn.preprocessing import StandardScaler
# Init, fit and transform

scaler = StandardScaler()

In [5]:
# the same, but in 1 line
scaler.fit_transform(X)

array([[-1., -1., -1.],
       [ 1.,  1.,  1.]])

#### 3. Pipeline

In [6]:
from sklearn.pipeline import Pipeline
# create a pipeline
pipeline = Pipeline([("scaler", StandardScaler()), ("lr",LinearRegression())])
# fit the whole pipeline
pipeline.fit(X, y)

# we can now use it like any other estimator and make a prediction
pipeline.predict(X_test)

array([3.08])

In [7]:
# init modela and transformer
lr = LinearRegression()
scaler = StandardScaler()

# Fit transformer
scaler.fit(X)

# Transform the data
X_scaled = scaler.transform(X)

# Fit the model
lr.fit(X_scaled, y)

# Predict
lr.predict(scaler.transform(X_test))

array([3.08])

## Part 2. Using sklearn pipeline

In [9]:
data = pd.read_csv('./data/house_prices_small.csv')
data.head()

,SalePrice,LotArea,OverallQual,SaleCondition,YearBuilt
0,208500,8450,7,Normal,2003
1,181500,9600,6,Normal,1976
2,223500,11250,7,Normal,2001
3,140000,9550,7,Abnorml,1915
4,250000,14260,8,Normal,2000


### 1. Prepare the data